In [1]:
'''
Try the following practice problems to test your understanding of the lab. Please note that the solutions for the following are not shared. You are encouraged to use the discussion forums in case you need help.

Modify the code to extract Film, Year, and Rotten Tomatoes' Top 100 headers.

Restrict the results to only the top 25 entries.

Filter the output to print only the films released in the 2000s (year 2000 included).
'''

"\nTry the following practice problems to test your understanding of the lab. Please note that the solutions for the following are not shared. You are encouraged to use the discussion forums in case you need help.\n\nModify the code to extract Film, Year, and Rotten Tomatoes' Top 100 headers.\n\nRestrict the results to only the top 25 entries.\n\nFilter the output to print only the films released in the 2000s (year 2000 included).\n"

In [2]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

c:\Users\USER\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [6]:
# Getting url
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'

# Using requests
r = requests.get(url).text
soup = BeautifulSoup(r, 'html.parser')

In [42]:
tables = soup.find_all('tbody')
rows = tables[0].find_all('tr')

### Notes
#### When I first ran this code, the dataframe was not populating. I did this code:

```
df = pd.DataFrame(columns=['Film','Year','RottenTomatoesRank'])
count = 0 

for row in rows:
    col = row.find_all('td')
    if len(col)!=0:
        data_dict = {'Film':col[1].contents[0], 'Year':col[2].contents[0], 'RottenTomatoesRank':col[3].contents[0]}
        df1 = pd.DataFrame(data_dict, index = [0])
        df = pd.concat([df,df1])
    else:
        break 
```

I have to be careful with the use of the ifs statements and where I place the break. Because I was basically saying, if len(col) != 0, break the loop. 

In [135]:
tables = soup.find_all('tbody')
rows = tables[0].find_all('tr')

# Film, Year, and Rotten Tomatoes' Top 100 headers

df = pd.DataFrame(columns=['Film','Year','RottenTomatoesRank'])
count = 0 

for row in rows:
    if count < 150: # This condition was preventing my code to populate the dataframe
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {'Film':col[1].contents[0], 'Year':col[2].contents[0], 'RottenTomatoesRank':col[3].contents[0]}
            df1 = pd.DataFrame(data_dict, index = [0])
            df = pd.concat([df,df1])
            count+=1
    else:
        break # The break with the condition was preventing my code to populate the dataframe 

In [141]:
# Dropping rows where "RottenTomatoesRank" is 'unranked'
df = df[df['RottenTomatoesRank'] != 'unranked']
df['RottenTomatoesRank'] = df['RottenTomatoesRank'].astype(int)
df = df.sort_values(by=['RottenTomatoesRank'],  ascending = True)

In [142]:
df

,Film,Year,RottenTomatoesRank
0,It Happened One Night,1934,1
0,Citizen Kane,1941,2
0,The Wizard of Oz,1939,3
0,Modern Times,1936,4
0,Black Panther,2018,5
0,Parasite,2019,6
0,Avengers: Endgame,2019,7
0,Casablanca,1942,8
0,Lady Bird,2017,12
0,Get Out,2017,15


In [145]:
database = 'Practice_Movies.db'
conn = sqlite3.connect(database)
df.to_sql('movie_data', conn, index = False, if_exists='replace')
conn.close()

#### Used SQLite3 extension for vs code to view the table.
SQLITE Explorer